<a href="https://colab.research.google.com/github/sowmya-520/NNDL/blob/main/NNDL_Assignment4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Assignment 4

Build a simple linear regression model using AdaGrad + SGD optimizer to help 33/11KV substation electric utility to trade power effectively in an day ahead energy market by estimating load ata particular hour based on the load at the same day and hour but in previous week. Dataset: Active power load dataset

• Write the python code to build simple linear regression model using Adagrad+SGD optimizer
• Do the data normalization
• Split the data for train and test (90:10)
• Train the simple linear regression model training data
• Compute MSE, RMSE and MAE with training data
• Compute MSE, RMSE and MAE with testing data

#Mount Google Drive

In [13]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#Import libraries

In [14]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

#Load the data

In [15]:
data = pd.read_excel('/content/drive/MyDrive/data3.xlsx')
data.head()

,Date,Time,Load (kW)
0,01.09.2018,00:00:00,5551.82208
1,NaN,01:00:00,4983.17184
2,NaN,02:00:00,4888.39680
3,NaN,03:00:00,5072.95872
4,NaN,04:00:00,5196.25980


In [16]:
x = []
y = []
for i in range(24 * 7, data.shape[0]):
    x.append(data['Load (kW)'].iloc[i - 24 * 7]) #Since x is 24 * 7 hours before y
    y.append(data['Load (kW)'].iloc[i])

In [17]:
data = pd.DataFrame({'Previous Week Load' : x, 'Present Week Load' : y})
data.head()

,Previous Week Load,Present Week Load
0,5551.82208,5346.52812
1,4983.17184,5138.89596
2,4888.39680,4940.77248
3,5072.95872,4766.18688
4,5196.25980,5020.11540


#Data Normalisation

In [18]:
maxX = np.max(data['Previous Week Load'])
minX = np.min(data['Previous Week Load'])
maxY = np.max(data['Present Week Load'])
minY = np.min(data['Present Week Load'])

data['Previous Week Load'] = (data['Previous Week Load'] - minX) / (maxX - minX)
data['Present Week Load'] = (data['Present Week Load'] - minY) / (maxY - minY)

#Split the Data

In [19]:
x = data['Previous Week Load']
y = data['Present Week Load']
trainX, testX, trainY, testY = train_test_split(x, y, test_size=0.1, random_state=101)

# AdaGrad + SGD optimizer

In [21]:
m = np.random.uniform(-2, 2)
c = np.random.uniform(-2, 2)
eta = 0.5
epochs = 500
gm2 = 0
gc2 = 0
e = 1E-6
for _ in range(epochs):
    for i in range(trainX.shape[0]):
        gradM = -1 * (trainY.iloc[i] - m * trainX.iloc[i] - c) * trainX.iloc[i]
        gradC = -1 * (trainY.iloc[i] - m * trainX.iloc[i] - c)
        gm2 += (gradM ** 2)
        gc2 += (gradC ** 2)
        m -= (eta * gradM) / ((e + gm2) ** 0.5)
        c -= (eta * gradC) / ((e + gc2) ** 0.5)
print(f'm = {m}\nc = {c}')

m = 0.48573487110831237
c = 0.23135537657784333


In [22]:
train_pred = [m * trainX.iloc[i] + c for i in range(trainX.shape[0])]
test_pred = [m * testX.iloc[i] + c for i in range(testX.shape[0])]

train_pred = [i * (maxY - minY) + minY for i in train_pred]
test_pred = [i * (maxY - minY) + minY for i in test_pred]
trainY = [i * (maxY - minY) + minY for i in trainY]
testY = [i * (maxY - minY) + minY for i in testY]

training_MAE = sum([abs(train_pred[i] - trainY[i]) for i in range(len(trainY))]) / len(trainY)
training_MSE = sum([(train_pred[i] - trainY[i]) ** 2 for i in range(len(trainY))]) / len(trainY)
training_RMSE = training_MSE ** 0.5
print('Training Error :')
print(f'MAE : {training_MAE}')
print(f'MSE : {training_MSE}')
print(f'RMSE : {training_RMSE}')

testing_MAE = sum([abs(test_pred[i] - testY[i]) for i in range(len(testY))]) / len(testY)
testing_MSE = sum([(test_pred[i] - testY[i]) ** 2 for i in range(len(testY))]) / len(testY)
testing_RMSE = testing_MSE ** 0.5
print('Testing Error :')
print(f'MAE : {testing_MAE}')
print(f'MSE : {testing_MSE}')
print(f'RMSE : {testing_RMSE}')

Training Error :
MAE : 737.0271778949027
MSE : 891424.8663201508
RMSE : 944.1529888318687
Testing Error :
MAE : 787.1173243676815
MSE : 968116.6346298852
RMSE : 983.929181714764


In [23]:
pd.DataFrame({'Actual' : trainY, 'Predicted' : train_pred})

,Actual,Predicted
0,5086.98792,5060.571139
1,6378.60960,5731.517132
2,7647.78456,6182.147977
3,6165.05400,5656.058436
4,5918.45184,5501.807528
...,...,...
1809,5020.11540,5866.676080
1810,4048.82712,5524.157242
1811,5674.03200,5578.175364
1812,5621.65632,6995.526045


In [24]:
x = float(input('Enter the load at same hour in previous week: '))
x = (x - minX) / (maxX - minX)
prediction = m * x + c
prediction = (prediction * (maxY - minY)) + minY
print('Predicted load at present hour :', prediction)

Enter the load at same hour in previous week: 5437.908
Predicted load at present hour : 5641.604565140473
